In [2]:
import pandas as pd
from tqdm import tqdm

In [3]:
essalud = pd.ExcelFile('data/EsSalud en Cifras_Definitivo Diciembre  2017.xlsx')

In [4]:
sheet_names = essalud.sheet_names[1:-8]

In [4]:
dfs = [pd.DataFrame(columns=['nivel-centro', 'periodo']+[sheet_name]) for sheet_name in sheet_names]
n_sheet = 0

for sheet_name in tqdm(sheet_names):
    df = essalud.parse(sheet_name=sheet_name, skiprows=13)
    df = df[df['Red'].isna()].drop(labels=['Red','Suma de Total'], axis=1)
    df['nivel_centro'] = df['nivel'] + '-' + df['Centro']
    df.index = df['nivel_centro']
    
    df_temp = df.iloc[:,2:-1]
    periods_names = ['per_'+ elem[-6:-2] + '-' + elem[-2:] for elem in df.columns[2:-1]]
    df_temp.columns = periods_names
    
    ix = 0
    for row in df_temp.itertuples():
        hospital = row.Index
        for period_n, value in enumerate(row[1:]):
            dfs[n_sheet].loc[ix,'nivel-centro'] = hospital
            dfs[n_sheet].loc[ix,'periodo'] = periods_names[period_n]
            dfs[n_sheet].loc[ix,sheet_name] = value
            ix += 1
    n_sheet += 1

100%|██████████| 78/78 [03:10<00:00,  2.44s/it]


In [5]:
for i in range(len(dfs)):
    dfs[i].index = dfs[i]['nivel-centro'] + '_' + dfs[i]['periodo']
    dfs[i] = dfs[i].drop(labels=['nivel-centro','periodo'], axis=1)

In [6]:
final_df = dfs[0]

for i in range(1,len(dfs)):
    final_df = final_df.join(dfs[i], how='outer')

In [8]:
final_df.to_csv('data/essalud_data_2017.csv')